In [ ]:
# Binary Tree Vertical Order Traversal
# here when values are at same level, we need to sort them in ascending order

class Solution:
    def verticalTraversal(self, root: Optional[TreeNode]) -> List[List[int]]:
        if root is None:
            return []
        q = deque([(root, 0)])
        d = defaultdict(list)
        while q:
            tq = deque()
            td = defaultdict(list)
            for _ in range(len(q)):
                root, offset = q.popleft()
                td[offset].append(root.val)
                if root.left:
                    tq.append((root.left, offset - 1))
                if root.right:
                    tq.append((root.right, offset + 1))
            for i in td:
                d[i].extend(sorted(td[i]))
            q = tq
        return [v for _, v in sorted(d.items())]


In [ ]:
# https://leetcode.com/problems/create-binary-tree-from-descriptions/description/

# Input: descriptions = [[20,15,1],[20,17,0],[50,20,1],[50,80,0],[80,19,1]]
# 1 - left child, 0 - right child
# Output: [50,20,80,15,17,19]

class Solution:
    def createBinaryTree(self, descriptions: List[List[int]]) -> Optional[TreeNode]:
        children = set()
        m = {}
        for p, c, l in descriptions:
            np = m.setdefault(p, TreeNode(p))
            nc = m.setdefault(c, TreeNode(c))
            if l:
                np.left = nc
            else:
                np.right = nc
            children.add(c)
        root = (set(m) - set(children)).pop()
        return m[root]

# create parent node and child node, then link them together, finally find the root node if it is not in the children set
class Solution:
    def createBinaryTree(self, descriptions: List[List[int]]) -> Optional[TreeNode]:
        mp = {}
        seen = set()
        for p, c, left in descriptions: 
            if p not in mp: mp[p] = TreeNode(p)
            if c not in mp: mp[c] = TreeNode(c)
            if left: mp[p].left = mp[c]
            else: mp[p].right = mp[c]
            seen.add(c)
        for p, _, _ in descriptions: 
            if p not in seen: return mp[p]

In [ ]:
# https://leetcode.com/problems/step-by-step-directions-from-a-binary-tree-node-to-another/description/
"""
Input: root = [5,1,2,3,null,6,4], startValue = 3, destValue = 6
Output: "UURL"
Explanation: The shortest path is: 3 → 1 → 5 → 2 → 6.
"""

# Find the lowest common ancestor of the two nodes, then traverse from the LCA to the start node and dest node
# from start node to LCA, all the path should be 'U', from LCA to dest node it should be traversed as it is

class Solution:
    def getDirections(self, root: Optional[TreeNode], startValue: int, destValue: int) -> str:
        def lca(root, start, dest):
            if root is None: return None
            if root.val == start or root.val == dest: return root
            la = lca(root.left, start, dest)
            ra = lca(root.right, start, dest)
            if (la and ra): return root
            return la if la else ra

        def traverse(root, path, val):
            if not root:
                return False

            if root.val == val:
                return True
            
            path.append('L')
            if traverse(root.left, path, val):
                return True  # If found, then return
            path.pop()  # If not found, then backtrack
            
            path.append('R')
            if traverse(root.right, path, val):
                return True  # If found, then return
            path.pop()  # If not found, then backtrack
            return False

        lca_root = lca(root, startValue, destValue)
        path_start, path_dest = [], []
        traverse(lca_root, path_start, startValue)
        traverse(lca_root, path_dest, destValue)
        for i in range(len(path_start)):
            path_start[i] = 'U'

        return "".join(path_start + path_dest)

In [ ]:
# https://leetcode.com/problems/delete-nodes-and-return-forest/description/

"""
Input: root = [1,2,3,4,5,6,7], to_delete = [3,5]
Output: [[1,2,null,4],[6],[7]]
"""

# first check if left and right child is in the delete set, if yes, then set it to None
# if the current node is in the delete set, then add its left and right child to the result
class Solution:
    def delNodes(self, root: Optional[TreeNode], to_delete: List[int]) -> List[TreeNode]:
        ans = []
        dq = deque()
        del_set = set(to_delete)
        if root is None: return ans
        if root.val not in del_set: ans.append(root)
        dq.append(root)
        while dq:
            curr = dq.popleft()
            if curr.left != None: dq.append(curr.left)
            if curr.right != None: dq.append(curr.right)
            if curr.left != None and curr.left.val in del_set:
                curr.left = None
            if curr.right != None and curr.right.val in del_set:
                curr.right = None
            if curr.val in del_set:
                if curr.left != None: ans.append(curr.left)
                if curr.right != None: ans.append(curr.right)

        return ans
        

In [ ]:
# https://leetcode.com/problems/number-of-good-leaf-nodes-pairs/description

"""
Input: root = [1,2,3,4,5,6,7], distance = 3
Output: 2
Explanation: The good pairs are [4,5] and [6,7] with shortest path = 2. The pair [4,6] is not good because the length of ther shortest path between them is 4.
"""
# use post order traversal to return the distances from the leaf nodes to the current node
# then use two for loops to find the good pairs

class Solution:
    def countPairs(self, root: TreeNode, distance: int) -> int:
        count = 0
        def dfs(node):
            nonlocal count
            if not node: return []
            if not node.left and not node.right: return [1]
            left = dfs(node.left)
            right = dfs(node.right)
            count += sum(l+r <= distance for l in left for r in right)
            return [n+1 for n in left + right if n+1 < distance]
        dfs(root)
        return count
        